# Tutorial and demo of core MEEGsim features

In this tutorial/demo, we will start with the pre-requisites for the simulation (source space / forward model), then look into building blocks currently provided by MEEGsim and how one can combine them in a simulation.

**NOTE:** this notebook is designed to run in full without you having to change anything in code apart from the path to the data (see [0. Configuration](#0-configuration) below). However, you're always welcome to explore and play around with the data. The **EXERCISES** part of sections below suggests some things you could try.

In [ ]:
import matplotlib.pyplot as plt
import mne
import numpy as np

from mne.datasets import fetch_fsaverage
from pathlib import Path

from meegsim.coupling import ppc_shifted_copy_with_noise
from meegsim.location import select_random
from meegsim.waveform import narrowband_oscillation, one_over_f_noise, white_noise
from meegsim.simulate import SourceSimulator

from meegsim_tutorial.utils import info_from_montage
from meegsim_tutorial.viz import show_sources, show_waveforms, show_leadfield

## 0. Configuration

Below, provide the same path to the sample dataset that you used during the installation check.

In [ ]:
download_path = "~/mne_data/"
subjects_dir = Path(download_path).expanduser().absolute() / "MNE-fsaverage-data"
# download_path = FILL_ME(
#     "Provide the same path (or use None) that used during the installation check."
# )

In [ ]:
fs_dir = fetch_fsaverage(subjects_dir=subjects_dir)

## 1. Navigating the source space

Source space is one of the key ingredients in every simulation, since it defines location and orientations of all sources that we model. In this section, we have a closer look at how one can navigate the source space. 

**Take home:** in MEEGsim, we use two numbers to define position of sources that are added to the simulation: 
 * index of the source space (0 - left hemisphere, 1 - right hemisphere) 
 * index of the vertex within the respective hemisphere (`vertno`).

Let's create a source space using the template MPI (`fsaverage`) from the sample MNE dataset and inspect it first. The `spacing` parameter below defines how coarse the source space. We recommend using `oct5` throughout the tutorial to reduce the computational load during the workshop. Find more about other possible and recommended spacing values [here](https://mne.tools/stable/documentation/cookbook.html#setting-up-the-source-space).

In [ ]:
subject = "fsaverage"
src = mne.setup_source_space(
    subject=subject, spacing="oct5", subjects_dir=subjects_dir, add_dist=False
)

If we print the resulting `src`, we get an overview of the generated source space: two hemispheres with 1026 vertices in each hemisphere. Hemisphere-specific data can be accessed with `src[0]` and  `src[1]` for left and right hemisphere, respectively.

In [ ]:
print(src)

In [ ]:
src[0]

It is also possible to visualize the positions of all sources on the brain surface. Each source is a dipole that corresponds to a group of aligned pyramidal neurons:

In [ ]:
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    surfaces="white",
    coord_frame="mri",
    src=src,
)

Each vertex has a unique number between 0 and 163841. The `vertno` array stored indices of all vertices that belong to the source space (in this case, the left hemisphere):

In [ ]:
src[0]["vertno"]

In MEEGsim, we use the combination of the hemisphere index (0/1) and `vertno` to define the position of added sources. You can try it out below in a small example (not yet simulation). The helper function `show_source` will show the source that is defined by `hemi_idx` and `vertno` below.

In [ ]:
hemi_idx = 0
vertno = 0

In [ ]:
# TIP: add surf="pial" or surf="pial_semi_inflated" to show sulci/gyri
brain = show_sources([(hemi_idx, vertno)], subjects_dir)

**EXERCISES**:
1. Try moving the source to the right hemisphere.
2. Try changing the `vertno` value to select a source in frontal/occipital/your favorite area.

Sometimes, it can be useful to choose vertices randomly, and the [`select_random`](https://meegsim.readthedocs.io/en/latest/generated/meegsim.location.select_random.html) function can be used for this purpose. Notice the format of its output (pairs of numbers):

In [ ]:
select_random(src, n=10)

In [ ]:
show_sources(select_random(src, n=10), subjects_dir)

## 2. Forward model

Forward model is the second key ingredient of all simulations. It provides a mapping between source- and sensor-space activity, which is described by the lead field matrix. In this section, we explore the lead field.

### 2.1. Channel locations

First, let's define channel locations for the simulated EEG setup. In MNE-Python, there are multiple [built-in channel configurations](https://mne.tools/stable/auto_tutorials/intro/40_sensor_locations.html#working-with-built-in-montages) (a.k.a. montages). In this tutorial, we will use the Biosemi64 montage with 64 channels, which is a common setup nowadays. Below, we first create an `mne.Info` object that describes channel locations and then plot them:

In [ ]:
info = info_from_montage("biosemi64")

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
info.plot_sensors(sphere="eeglab", axes=ax);

### 2.2. Lead field

With channel locations fixed, we are now ready to obtain the lead field. For this purpose, we use a pre-computed BEM (boundary element method) model of the template `fsaverage` head:

In [ ]:
bem = subjects_dir / subject / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif"
trans = subjects_dir / subject / "bem" / "fsaverage-trans.fif"

In [ ]:
fwd = mne.make_forward_solution(
    info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=None,
    verbose=True,
)

By default, the forward model allows arbitrary orientations of sources. However, MEEGsim at the moment only supports fixed orientations along the normal to the cortical surface, so we need to convert the forward solution accordingly:

In [ ]:
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
L = fwd["sol"]["data"]
print(L.shape)

Matrix `L` defined above is the resulting lead field. It has 64 rows (1 row per channel) and 2052 columns (1 column per modeled source). Lead field describes how all sources get mixed when their activity is projected to sensor space. We can explore the lead field of separate sources to get a better feeling of the mixing process:

In [ ]:
hemi_idx = 0
vertno = 0

In [ ]:
show_sources([(hemi_idx, vertno)], subjects_dir)

In [ ]:
fig = show_leadfield(fwd, info, hemi_idx, vertno)

**EXERCISE**: try changing `hemi_idx` and `vertno` and see how the lead field changes.

## 3. Generating source activity

With source space and lead field in place, we are now ready to start simulating data. In this section, we will introduce basic waveforms of source activity - noise (white or 1/f) and oscillations. All waveforms are generated from white noise by processing (e.g., filtering) the signal to obtain the desired properties.

Simulation of activity requires a vector of time points for each generated sample (similar to `raw.times` in MNE-Python). Below, we create such vector for 60 s of data with the sampling frequency of 250 Hz:

In [ ]:
sfreq = 250
duration = 60
times = np.arange(sfreq * duration) / sfreq

In [ ]:
times[:10]

### 3.1. Noise: background (1/f) and sensor (white)

In [ ]:
n1 = one_over_f_noise(1, times, slope=1)
n2 = one_over_f_noise(1, times, slope=1.5)
n3 = white_noise(1, times)

In [ ]:
fig = show_waveforms(np.vstack([n1, n2, n3]), times, n_seconds=2)

**NOTE:** if you re-run, all cells of section 3.1, the resulting time series change randomly. This can be helpful when generating multiple datasets (i.e., "subjects") for the same simulation idea.

### 3.2. Oscillatory activity

In [ ]:
s1 = narrowband_oscillation(1, times, fmin=8, fmax=12)
s2 = narrowband_oscillation(1, times, fmin=16, fmax=24)

In [ ]:
fig = show_waveforms(np.vstack([s1, s2]), times, n_seconds=2)

## 4. Combining the ingredients

By now, we learned how to place sources and generate their activity, but that doesn't feel like a proper simulation yet. It's time to combine the ingredients. For this purpose, we will use the `SourceSimulator` class provided by MEEGsim, which allows one to add sources (`add_point_sources`, `add_noise_sources`) to the simulation. When adding sources, we need to set their `location` and `waveform` either explicitly (`(hemi_idx, vertno)` and time course) or through a generating function (`select_random`, `one_over_f_noise`):

In [ ]:
sim = SourceSimulator(src)
sim.add_point_sources(
    location=[(0, 0), (1, 0)],
    waveform=np.vstack([s1, s2]),  # re-using results from section 3.2 here
    names=["m1-lh", "m1-rh"],
)
_ = sim.add_noise_sources(
    location=select_random,
    location_params=dict(n=100),
    waveform=one_over_f_noise,  # default, can be omitted
    waveform_params=dict(slope=1),  # default, can be omitted
)

The `sim` object does not contain the simulated data, it only describes how the sources should be simulated. By running its `simulate` method, we obtain a `SourceConfiguration` object which actually contains all sources and their data:

In [ ]:
sc = sim.simulate(sfreq=sfreq, duration=duration, fwd=fwd, random_state=123)

Each source has a name for quick access, and names can be set when creating the sources (see the `add_point_sources` call above; it is also helpful when defining ground-truth connectivity):

In [ ]:
sc._sources

If the names are not provided explicitly, they are (for now, this behavior might be changed) generated automatically:

In [ ]:
sc._noise_sources

### 4.2. Inspect and debug the source configuration

To get a better feeling of what we have just achieved, we can inspect the resulting source configuration in more detail. First, let's plot all sources:

In [ ]:
sc.plot(
    subject="fsaverage",
    subjects_dir=subjects_dir,
    colors=dict(point="red"),
    hemi="split",
    views=["lat", "med"],
    size=600,
    time_viewer=False,
)

In addition, we can access each non-noise source by its name and, for example, plot its activity:

In [ ]:
sc["m1-lh"]

In [ ]:
fig = show_waveforms(sc["m1-lh"].waveform, sc.times)

**EXERCISE**: plot the activity of the source `m1-rh`.

### 4.3. Obtain data

Finally, we can also obtain the simulated data in source and sensor space with `to_stc` and `to_raw` methods, respectively:

In [ ]:
stc = sc.to_stc()

In [ ]:
stc

To obtain sensor-space data, we need to provide the forward model (`fwd`) and channel locations (`info`). In addition, we can add a certain level of measurement noise (`sensor_noise_level`):

In [ ]:
raw = sc.to_raw(fwd, info, sensor_noise_level=0.01)

Let's inspect the simulated data and its spectra (**NOTE** `%matplotlib qt` forces the plots to open in pop-up window, use `%matplotlib inline` to disable it):

In [ ]:
%matplotlib qt
raw.plot(scalings=dict(eeg=2e-6))

In [ ]:
%matplotlib inline
fig = raw.compute_psd(fmax=40, n_fft=2 * sfreq, n_overlap=sfreq).plot(sphere="eeglab")

### 4.3. Adjusting the signal-to-noise ratio (SNR)

In the plot above, oscillatory activity is present but not very pronounced. To control its level, we can adjust the SNR of oscillatory activity relative to 1/f noise. This has to be done when simulating the data so we redo this step below. The desired SNR is specified in the `snr_global` argument, while `snr_params` define the frequency band for the adjustment of SNR:

In [ ]:
sc = sim.simulate(
    sfreq=sfreq,
    duration=duration,
    snr_global=3,
    snr_params=dict(fmin=8, fmax=30),
    fwd=fwd,
    random_state=123,
)
raw = sc.to_raw(fwd, info, sensor_noise_level=0.01)

In [ ]:
fig = raw.compute_psd(fmax=40, n_fft=2 * sfreq, n_overlap=sfreq).plot(sphere="eeglab")

**EXERCISE**: try other values of `snr_global` to get the spectra which look reasonable to you.

## 5. Setting up ground-truth connectivity

## 6. Summary

Congrats, you've made it to the end of the demo! Below you can find an overview of what we've covered so far. This rather short script generates simulated EEG data for 100 sources of 1/f activity and 2 sources of alpha activity with pre-defined coupling parameters and desired SNR. Comments above function calls highlight the similarity between MEEGsim syntax and textual description that one could use in the paper.

In [ ]:
sim = SourceSimulator(src)

# 100 noise sources placed randomly
sim.add_noise_sources(location=select_random, location_params=dict(n=100))

# 2 sources of narrowband (8-12 Hz) oscillation
sim.add_point_sources(
    location=[(0, 0), (1, 0)],
    waveform=narrowband_oscillation,
    waveform_params=dict(fmin=8, fmax=12),
    names=["m1-lh", "m1-rh"],
)

# Coupling between alpha sources with a phase lag of pi/2 and coherence of 0.5
sim.set_coupling(
    ("m1-lh", "m1-rh"),
    method=ppc_shifted_copy_with_noise,
    fmin=8,
    fmax=12,
    coh=0.5,
    phase_lag=np.pi / 2,
)

# SNR of 0.5 in 8-30 Hz
sc = sim.simulate(
    sfreq=sfreq,
    duration=duration,
    snr_global=0.5,
    snr_params=dict(fmin=8, fmax=30),
    fwd=fwd,
    random_state=123,
)

# 1% of sensor space noise
raw = sc.to_raw(fwd, info, sensor_noise_level=0.01)

![That's all, folks!](../assets/fin.png)